## Simple Chase example

In [ ]:
# https://pettingzoo.farama.org/environments/mpe/simple_push/
from pettingzoo.mpe import simple_tag_v3

env = simple_tag_v3.env(num_good=1, num_adversaries=1 , render_mode="human")
env.reset(seed=42)
verbose = False

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    if verbose == True:

        print("Agent: ", agent)
        print("===============================================")
        print("Observation: ", observation)
        print("===============================================")
        print("Action:", action)
        print("===============================================")
        print("reward: ", reward)
        print("===============================================")

    

    env.step(action)

env.close()

Agent:  adversary_0
Observation:  [ 0.          0.          0.5479121  -0.12224312 -1.2783929   0.97836334
 -0.07786063  0.6371589   0.16928375  0.51697916  0.          0.        ]
Action: 2
reward:  0.0
Agent:  agent_0
Observation:  [ 0.          0.          0.71719587  0.39473605 -1.4476767   0.46138418
 -0.24714437  0.12017969 -0.16928375 -0.51697916]
Action: 1
reward:  0.0
Agent:  adversary_0
Observation:  [ 3.0000001e-01 -0.0000000e+00  5.4791212e-01 -1.2224312e-01
 -1.2783929e+00  9.7836334e-01 -7.7860631e-02  6.3715887e-01
  1.6928375e-01  5.1697916e-01 -4.0000001e-01 -7.3046807e-14]
Action: 3
reward:  0.0
Agent:  agent_0
Observation:  [-4.0000001e-01 -7.3046807e-14  7.1719587e-01  3.9473605e-01
 -1.4476767e+00  4.6138418e-01 -2.4714437e-01  1.2017969e-01
 -1.6928375e-01 -5.1697916e-01]
Action: 2
reward:  0.0
Agent:  adversary_0
Observation:  [ 2.2499999e-01 -3.0000001e-01  5.7791209e-01 -1.2224312e-01
 -1.3083929e+00  9.7836334e-01 -1.0786063e-01  6.3715887e-01
  9.9283740e-02 

In [4]:
from pettingzoo.mpe import simple_adversary_v3

env = simple_adversary_v3.parallel_env(render_mode="human", N=1)
observations, infos = env.reset()
verbose = True
print(env.agents)

while env.agents:
    # this is where you would insert your policy
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}

    observations, rewards, terminations, truncations, infos = env.step(actions)
    
    if verbose == True:

        # print("Agent: ", agent)
        # print("===============================================")
        print("Observation: ", observations)
        print("===============================================")
        print("Action:", actions)
        print("===============================================")
        print("reward: ", rewards)
        print("===============================================")

    
env.close()


['adversary_0', 'agent_0']
Observation:  {'adversary_0': array([ 0.43422684,  0.42705926, -1.3700244 ,  0.6611358 ], dtype=float32), 'agent_0': array([ 1.8042513 , -0.23407649,  1.8042513 , -0.23407649,  1.3700244 ,
       -0.6611358 ], dtype=float32)}
Action: {'adversary_0': np.int64(2), 'agent_0': np.int64(1)}
reward:  defaultdict(<class 'int'>, {'adversary_0': -0.6090423301268665, 'agent_0': -1.2103296517775446})
Observation:  {'adversary_0': array([ 0.38422683,  0.42705926, -1.4700245 ,  0.6611358 ], dtype=float32), 'agent_0': array([ 1.8542513 , -0.23407649,  1.8542513 , -0.23407649,  1.4700245 ,
       -0.6611358 ], dtype=float32)}
Action: {'adversary_0': np.int64(2), 'agent_0': np.int64(3)}
reward:  defaultdict(<class 'int'>, {'adversary_0': -0.5744648616245545, 'agent_0': -1.2945026432914941})
Observation:  {'adversary_0': array([ 0.29672682,  0.42705926, -1.5950245 ,  0.6111358 ], dtype=float32), 'agent_0': array([ 1.8917513 , -0.18407649,  1.8917513 , -0.18407649,  1.5950245 

In [ ]:
env = simple_adversary_v3.parallel_env(render_mode="human", N=1)
env.reset()
print(env.agents)

['adversary_0', 'agent_0']


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pettingzoo.mpe import simple_tag_v3
from collections import defaultdict
from tqdm import tqdm

env = simple_tag_v3.env(num_good=1, num_adversaries=1, render_mode="rgb_array")
env.reset(seed=42)


# --- Independent Q-Learning (IQL) example skeleton ---
alpha = 0.01
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.05
epsilon_decay_steps = 80000
decay_rate = (epsilon - epsilon_min) / epsilon_decay_steps

Q_tables = {agent: defaultdict(lambda: np.zeros(env.action_space(agent).n))
            for agent in env.agents}

def get_state(obs):
    return tuple(np.round(obs, 1))  # discretize continuous obs

returns = []
for t in tqdm(range(10)):
    env.reset()
    total_reward = 0
    done = {a: False for a in env.agents}

    while not all(done.values()):
        for agent in env.agent_iter():
            obs, reward, term, trunc, info = env.last()
            done[agent] = term or trunc
            if done[agent]:
                env.step(None)
                continue
            state = get_state(obs)
            if np.random.rand() < epsilon:
                action = env.action_space(agent).sample()
            else:
                action = np.argmax(Q_tables[agent][state])

            env.step(action)
            next_obs, reward, term, trunc, _ = env.last()
            next_state = get_state(next_obs)
            done[agent] = term or trunc

            best_next = np.max(Q_tables[agent][next_state])
            Q_tables[agent][state][action] += alpha * (reward + gamma * best_next - Q_tables[agent][state][action])
            total_reward += reward

    returns.append(total_reward)

    # Decay epsilon
    if t < epsilon_decay_steps:
        epsilon -= decay_rate
        epsilon = max(epsilon, epsilon_min)

# Plot smoothed curve
window = 1000
smoothed = np.convolve(returns, np.ones(window)/window, mode='valid')
plt.plot(smoothed)
plt.xlabel('Environment time steps')
plt.ylabel('Evaluation returns')
plt.show()


## Joint Action Learning with Agent Modeling using Deep Q-Networks (DQN)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pettingzoo.mpe import simple_tag_v3
from collections import defaultdict
from tqdm import trange
import torch 
from torch import nn
from torch.nn import functional as F
import collections

env = simple_tag_v3.parallel_env(num_good=1, num_adversaries=1, render_mode=None)
env.reset(seed=42)

alpha = 0.01
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.05
epsilon_decay_steps = 80000
decay_rate = (epsilon - epsilon_min) / epsilon_decay_steps
episodes = 1000
BUFFER_CAPACITY = 1000
BATCH_SIZE = 64
GAMMA = 0.99       # Discount factor
TAU = 0.005        # For soft target network updates

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

/tmp/ipykernel_6220/1541967011.py:3: DeprecationWarning: The environment `pettingzoo.mpe` has been moved to `mpe2` and will be removed in a future release.Please update your imports.
  from pettingzoo.mpe import simple_tag_v3


Using device: cuda


In [2]:
# env.observation_space('adversary_0').shape[0]
obs, info = env.reset()
np.concat((obs['adversary_0'], obs['agent_0']))

array([ 0.        ,  0.        , -0.74377275, -0.09922812,  1.0027299 ,
        0.680199  ,  0.6419183 , -0.39174217,  0.4853688 ,  0.9527581 ,
        0.        ,  0.        ,  0.        ,  0.        , -0.25840396,
        0.85353   ,  0.51736116, -0.27255908,  0.15654951, -1.3445003 ,
       -0.4853688 , -0.9527581 ], dtype=float32)

In [3]:
## Deep Q-Network (DQN)

class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    def forward(self, x):
        return self.net(x)

class ModelNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    def forward(self, x):
        return F.softmax(self.net(x), dim=-1)

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, next_states, dones = zip(*[self.buffer[idx] for idx in batch])
        return (np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones))

    def __len__(self):
        return len(self.buffer)


In [ ]:

# --- Joint Action Learning with Agent Modeling using Deep Q-Networks (DQN) ---

global_state_dim = env.observation_space('adversary_0').shape[0] + env.observation_space('agent_0').shape[0]
action_dim_0 = env.action_space('agent_0').n
action_dim_1 = env.action_space('adversary_0').n
joint_action_dim = action_dim_0 * action_dim_1



# --- Agent 0 (Agent_0) ---
# Q-Net learns Q_0(s, a_0, a_1)
# agent_net = QNetwork(global_state_dim, joint_action_dim)
# agent_target_net = QNetwork(global_state_dim, joint_action_dim)
# agent_target_net.load_state_dict(agent_net.state_dict())

# Model-Net learns pi_1(a_1 | s)
# It models Agent 1 (the opponent)
# agent_model = ModelNetwork(global_state_dim, action_dim_1) 

# --- Agent 0 (Agent_0) ---
agent_net = QNetwork(global_state_dim, joint_action_dim).to(device)
agent_target_net = QNetwork(global_state_dim, joint_action_dim).to(device)
agent_target_net.load_state_dict(agent_net.state_dict())
agent_model = ModelNetwork(global_state_dim, action_dim_1).to(device) 



# --- Agent 1 (Adversary_0) ---
# Q-Net learns Q_1(s, a_0, a_1)
# adversary_net = QNetwork(global_state_dim, joint_action_dim)
# adversary_target_net = QNetwork(global_state_dim, joint_action_dim)
# adversary_target_net.load_state_dict(adversary_net.state_dict())

# Model-Net learns pi_0(a_0 | s)
# It models Agent 0 (the opponent)
# adversary_model = ModelNetwork(global_state_dim, action_dim_0)

adversary_net = QNetwork(global_state_dim, joint_action_dim).to(device)
adversary_target_net = QNetwork(global_state_dim, joint_action_dim).to(device)
adversary_target_net.load_state_dict(adversary_net.state_dict())
adversary_model = ModelNetwork(global_state_dim, action_dim_0).to(device)




agent_buffer = ReplayBuffer(BUFFER_CAPACITY)
adversary_buffer = ReplayBuffer(BUFFER_CAPACITY)

q_optimizer_0 = torch.optim.Adam(agent_net.parameters(), lr=alpha)
q_optimizer_1 = torch.optim.Adam(adversary_net.parameters(), lr=alpha)

model_optimizer_0 = torch.optim.Adam(agent_model.parameters(), lr=alpha)
model_optimizer_1 = torch.optim.Adam(adversary_model.parameters(), lr=alpha)

model_loss_fn = nn.NLLLoss()

In [61]:
from tqdm import tqdm

recent_rewards_agent_0 = collections.deque(maxlen=100)
recent_rewards_agent_1 = collections.deque(maxlen=100)
progress_bar = tqdm(range(episodes))
for episode in progress_bar:
    episode_reward_agent_0 = 0
    episode_reward_agent_1 = 0
    cumulative_q_loss_0 = 0
    cumulative_q_loss_1 = 0
    cumulative_model_loss_0 = 0
    cumulative_model_loss_1 = 0
    
    episode_steps = 0
    num_train_steps = 0 # To average loss correctly

    done = False
    obs, info = env.reset()
    while not done:
        global_state = np.concatenate((obs['adversary_0'], obs['agent_0']))
        
        state_tensor = torch.FloatTensor(global_state).unsqueeze(0).to(device)

        with torch.no_grad(): 
            # Get Q_0(s, a_0, a_1) for ALL joint actions
            # Output shape: [1, joint_action_dim]
            all_joint_q_agent = agent_net(state_tensor) 

            # Get pi_1(a_1 | s) -> Agent 0's model of Agent 1
            # Output shape: [1, action_dim_1]
            probs_opponent_1 = agent_model(state_tensor) 

            # Reshape Q-values into a matrix: [action_dim_0, action_dim_1]
            # Each row 'a0' contains Q-values for all opponent actions 'a1'
            q_matrix_agent = all_joint_q_agent.view(action_dim_0, action_dim_1)

            # Reshape opponent probs into a column vector: [action_dim_1, 1]
            probs_opponent_1_vec = probs_opponent_1.view(action_dim_1, 1)

            # This is Equation 6.17: AV_0 = Q_0 * pi_1
            # (dim_0, dim_1) @ (dim_1, 1) -> (dim_0, 1)
            action_values_agent_tensor = torch.matmul(q_matrix_agent, probs_opponent_1_vec)
            
            # Squeeze to get a 1D vector of action values for Agent 0
            # Shape: [action_dim_0]
            action_values_agent = action_values_agent_tensor.squeeze()

        with torch.no_grad():
            # Get Q_1(s, a_0, a_1) for ALL joint actions
            # Output shape: [1, joint_action_dim]
            all_joint_q_adversary = adversary_net(state_tensor)
            
            # Get pi_0(a_0 | s) -> Agent 1's model of Agent 0
            # Output shape: [1, action_dim_0]
            probs_opponent_0 = adversary_model(state_tensor)

            # Reshape Q-values into a matrix: [action_dim_0, action_dim_1]
            q_matrix_adversary = all_joint_q_adversary.view(action_dim_0, action_dim_1)

            # Reshape *this* opponent's probs (Agent 0) into a column vector: [action_dim_0, 1]
            probs_opponent_0_vec = probs_opponent_0.view(action_dim_0, 1)

            # This is Equation 6.17 for Agent 1: AV_1 = (Q_1^T) * pi_0
            # The sum is over a_0, so we must transpose the Q-matrix first.
            # (dim_1, dim_0) @ (dim_0, 1) -> (dim_1, 1)
            action_values_adversary_tensor = torch.matmul(q_matrix_adversary.T, probs_opponent_0_vec)
            
            # Squeeze to get a 1D vector of action values for Agent 1
            # Shape: [action_dim_1]
            action_values_adversary = action_values_adversary_tensor.squeeze()


        if np.random.rand() < epsilon:
            agent_action = np.random.choice(action_dim_0)
            adversary_action = np.random.choice(action_dim_1)
        else:
            agent_action = torch.argmax(action_values_agent).item()
            adversary_action = torch.argmax(action_values_adversary).item()

        actions = {'adversary_0': adversary_action, "agent_0": agent_action}
        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        done = all(terminations.values()) or all(truncations.values())
        global_next_obs = np.concatenate((next_obs['adversary_0'], next_obs['agent_0']))
        episode_reward_agent_0 += rewards['agent_0']
        episode_reward_agent_1 += rewards['adversary_0']
        episode_steps += 1

        # === 1. TRAIN AGENT MODELS (Line 11) ===
       
        # We use NLLLoss (Negative Log Likelihood Loss) because your
        # ModelNetwork outputs probabilities (after softmax).


        # --- Train agent_model (Agent 0's model of Agent 1) ---
        model_optimizer_0.zero_grad()
        probs_opponent_1 = agent_model(state_tensor)
        log_probs_1 = torch.log(probs_opponent_1 + 1e-9)
        target_action_1 = torch.LongTensor([adversary_action]).to(device)

        model_loss_0 = model_loss_fn(log_probs_1, target_action_1)
        cumulative_model_loss_0 += model_loss_0.item()
        model_loss_0.backward()
        model_optimizer_0.step()


        model_optimizer_1.zero_grad()
        probs_opponent_0 = adversary_model(state_tensor)
        log_probs_0 = torch.log(probs_opponent_0 + 1e-9)
        target_action_0 = torch.LongTensor([agent_action]).to(device)

        model_loss_1 = model_loss_fn(log_probs_0, target_action_0)
        cumulative_model_loss_1 += model_loss_1.item()
        model_loss_1.backward()
        model_optimizer_1.step()

        # =================================================================
        # === 2. UPDATE BUFFERS (Line 9) ===
        # We must convert the two separate actions into one joint_action index
        # to store in the buffer, as this is what our Q-network learns.
        # Formula: (a0 * num_actions_1) + a1
        joint_action = agent_action * action_dim_1 + adversary_action
        
        agent_buffer.push(global_state, joint_action, rewards['agent_0'], global_next_obs, done)
        
        adversary_buffer.push(global_state, joint_action, rewards['adversary_0'], global_next_obs, done)

        # =================================================================
        # === 3. TRAIN Q-NETWORKS (Line 12) ===
        # This is the Bellman update, using the AV(s') as the target.
        
        q_loss_fn = nn.MSELoss()

        if len(agent_buffer) >= BATCH_SIZE:
            states_b, joint_actions_b, rewards_b, next_states_b, dones_b = agent_buffer.sample(BATCH_SIZE)
            
            states_b = torch.FloatTensor(states_b).to(device)
            joint_actions_b = torch.LongTensor(joint_actions_b).unsqueeze(1).to(device) # Shape: [B, 1]
            rewards_b = torch.FloatTensor(rewards_b).unsqueeze(1).to(device)      # Shape: [B, 1]
            next_states_b = torch.FloatTensor(next_states_b).to(device)
            dones_b = torch.FloatTensor(dones_b).unsqueeze(1).to(device)          # Shape: [B, 1]

            with torch.no_grad():
                q_next_target_0 = agent_target_net(next_states_b) # [B, joint_action_dim]
                # Get pi_1(a_1 | s') from the *opponent model*
                model_next_1 = agent_model(next_states_b) # [B, action_dim_1]
                
                # Reshape for batch matrix multiplication
                q_matrix_next_0 = q_next_target_0.view(BATCH_SIZE, action_dim_0, action_dim_1)
                model_vec_next_1 = model_next_1.unsqueeze(2) # Shape: [B, action_dim_1, 1]
                
                # AV_0(s', a_0) = Q_target_0 * pi_1
                av_next_0 = torch.bmm(q_matrix_next_0, model_vec_next_1) # Shape: [B, action_dim_0, 1]
                
                # max_a0 AV_0(s', a_0)
                max_av_next_0, _ = torch.max(av_next_0, dim=1) # Shape: [B, 1]
                
                # Bellman Target: y_0 = r + gamma * max_AV * (1 - done)
                target_q_0 = rewards_b + (GAMMA * max_av_next_0 * (1 - dones_b))

            # --- Get Current Q-value: Q_0(s, a) ---
            q_current_0 = agent_net(states_b) # [B, joint_action_dim]
            # Get Q_0 for the *specific* joint action taken from the buffer
            q_current_0_selected = q_current_0.gather(1, joint_actions_b) # [B, 1]
            
            # --- Calculate Loss and Optimize ---
            q_loss_0 = q_loss_fn(q_current_0_selected, target_q_0)
            q_optimizer_0.zero_grad()
            cumulative_q_loss_0 += q_loss_0.item()
            num_train_steps += 1 # Increment this *only* when you train
            q_loss_0.backward()
            q_optimizer_0.step()

            # --- Soft update target network ---
            for target_param, local_param in zip(agent_target_net.parameters(), agent_net.parameters()):
                target_param.data.copy_(TAU * local_param.data + (1.0 - TAU) * target_param.data)
            
        # --- Train Agent 1's Q-Network (Adversary) ---
        if len(adversary_buffer) >= BATCH_SIZE:
            # Sample a batch
            states_b, joint_actions_b, rewards_b, next_states_b, dones_b = adversary_buffer.sample(BATCH_SIZE)

            # Convert to Tensors
            states_b = torch.FloatTensor(states_b).to(device)
            joint_actions_b = torch.LongTensor(joint_actions_b).unsqueeze(1).to(device)
            rewards_b = torch.FloatTensor(rewards_b).unsqueeze(1).to(device)
            next_states_b = torch.FloatTensor(next_states_b).to(device)
            dones_b = torch.FloatTensor(dones_b).unsqueeze(1).to(device)

            # --- Calculate Target y_1 ---
            # y_1 = r + gamma * max_a1' AV_1(s', a_1')
            with torch.no_grad():
                # Get Q_target_1(s', a_0, a_1) from the *target network*
                q_next_target_1 = adversary_target_net(next_states_b) # [B, joint_action_dim]
                # Get pi_0(a_0 | s') from the *opponent model*
                model_next_0 = adversary_model(next_states_b) # [B, action_dim_0]
                
                # Reshape for batch matrix multiplication
                q_matrix_next_1 = q_next_target_1.view(BATCH_SIZE, action_dim_0, action_dim_1)
                model_vec_next_0 = model_next_0.unsqueeze(2) # [B, action_dim_0, 1]
                
                # AV_1(s', a_1) = (Q_target_1^T) * pi_0
                # We transpose the Q-matrix to sum over a_0
                av_next_1 = torch.bmm(q_matrix_next_1.transpose(1, 2), model_vec_next_0) # [B, action_dim_1, 1]
                
                # max_a1 AV_1(s', a_1)
                max_av_next_1, _ = torch.max(av_next_1, dim=1) # [B, 1]
                
                # Bellman Target: y_1 = r + gamma * max_AV * (1 - done)
                target_q_1 = rewards_b + (GAMMA * max_av_next_1 * (1 - dones_b))
                
            # --- Get Current Q-value: Q_1(s, a) ---
            q_current_1 = adversary_net(states_b)
            q_current_1_selected = q_current_1.gather(1, joint_actions_b)
            
            # --- Calculate Loss and Optimize ---
            q_loss_1 = q_loss_fn(q_current_1_selected, target_q_1)
            q_optimizer_1.zero_grad()
            cumulative_q_loss_1 += q_loss_1.item()
            q_loss_1.backward()
            q_optimizer_1.step()

            # --- Soft update target network ---
            for target_param, local_param in zip(adversary_target_net.parameters(), adversary_net.parameters()):
                target_param.data.copy_(TAU * local_param.data + (1.0 - TAU) * target_param.data)

        # =================================================================
        # === 4. PREPARE FOR NEXT STEP ===
        obs = next_obs
        global_state = global_next_obs
    
    # --- (End of `while not done` loop) ---
    avg_q_loss_0 = cumulative_q_loss_0 / num_train_steps if num_train_steps > 0 else 0
    avg_q_loss_1 = cumulative_q_loss_1 / num_train_steps if num_train_steps > 0 else 0
    
    avg_model_loss_0 = cumulative_model_loss_0 / episode_steps if episode_steps > 0 else 0
    avg_model_loss_1 = cumulative_model_loss_1 / episode_steps if episode_steps > 0 else 0

    recent_rewards_agent_0.append(episode_reward_agent_0)
    recent_rewards_agent_1.append(episode_reward_agent_1)
    
    avg_reward_0 = sum(recent_rewards_agent_0) / len(recent_rewards_agent_0)
    avg_reward_1 = sum(recent_rewards_agent_1) / len(recent_rewards_agent_1)
    
    progress_bar.set_postfix(
        Avg_Rwd_A0=f"{avg_reward_0:.2f}",
        Avg_Rwd_A1=f"{avg_reward_1:.2f}",
        Epsilon=f"{epsilon:.3f}",
        Avg_Q_Loss=f"{avg_q_loss_0:.4f}", # Added a bit more precision
        Avg_M_Loss=f"{avg_model_loss_0:.4f}"
    )
    # tqdm.write(desc) # Use tqdm.write instead of print to avoid breaking the bar

    # Decay epsilon (your existing code)
    if episode < epsilon_decay_steps:
        epsilon -= decay_rate
        epsilon = max(epsilon, epsilon_min)


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [05:31<00:00,  3.01it/s, Avg_M_Loss=1.6480, Avg_Q_Loss=3.4614, Avg_Rwd_A0=-12.86, Avg_Rwd_A1=0.70, Epsilon=0.988]


### Visualize results

In [10]:
env = simple_tag_v3.parallel_env(num_good=1, num_adversaries=1, render_mode='human')
env.reset(seed=42)


global_state_dim = env.observation_space('adversary_0').shape[0] + env.observation_space('agent_0').shape[0]
action_dim_0 = env.action_space('agent_0').n
action_dim_1 = env.action_space('adversary_0').n
joint_action_dim = action_dim_0 * action_dim_1


agent_net = QNetwork(global_state_dim, joint_action_dim)
adversary_net = QNetwork(global_state_dim, joint_action_dim)
agent_model = ModelNetwork(global_state_dim, action_dim_1)
adversary_model = ModelNetwork(global_state_dim, action_dim_0)


agent_net.load_state_dict(torch.load("models/agent_net_best_model.pth"))
adversary_net.load_state_dict(torch.load("models/adversary_net_best_model.pth"))
agent_model.load_state_dict(torch.load("models/agent_model_best_model.pth"))
adversary_model.load_state_dict(torch.load("models/adversary_model_best_model.pth"))
### Visualize results


visualization_episodes = 3

for i in range(visualization_episodes):
    done = False
    obs, info = env.reset()
    while not done:
        global_state = np.concatenate((obs['adversary_0'], obs['agent_0']))
        
        state_tensor = torch.FloatTensor(global_state).unsqueeze(0)

        with torch.no_grad(): 
            all_joint_q_agent = agent_net(state_tensor) 
            probs_opponent_1 = agent_model(state_tensor) 
            q_matrix_agent = all_joint_q_agent.view(action_dim_0, action_dim_1)
            probs_opponent_1_vec = probs_opponent_1.view(action_dim_1, 1)
            action_values_agent_tensor = torch.matmul(q_matrix_agent, probs_opponent_1_vec)
            action_values_agent = action_values_agent_tensor.squeeze()

        with torch.no_grad():
            all_joint_q_adversary = adversary_net(state_tensor)
            probs_opponent_0 = adversary_model(state_tensor)
            q_matrix_adversary = all_joint_q_adversary.view(action_dim_0, action_dim_1)
            probs_opponent_0_vec = probs_opponent_0.view(action_dim_0, 1)
            action_values_adversary_tensor = torch.matmul(q_matrix_adversary.T, probs_opponent_0_vec)
            action_values_adversary = action_values_adversary_tensor.squeeze()

        if np.random.rand() < epsilon:
            agent_action = np.random.choice(action_dim_0)
            adversary_action = np.random.choice(action_dim_1)
        else:
            agent_action = torch.argmax(action_values_agent).item()
            adversary_action = torch.argmax(action_values_adversary).item()

        actions = {'adversary_0': adversary_action, "agent_0": agent_action}
        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        done = all(terminations.values()) or all(truncations.values())
        print(f"Rewards: Agent_0: {rewards['agent_0']}, Adversary_0: {rewards['adversary_0']}")

env.close()
        

Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0
Rewards: Agent_0: 0.0, Adversary_0: 0.0


## Soccer Environment

In [ ]:
import numpy as np
import random 

class SoccerEnv:
    def __init__(self, grid_size=(4, 5), gamma=0.9, draw_prob=0.1):
        self.rows, self.cols = grid_size
        self.gamma = gamma
        self.draw_prob = draw_prob
        self.actions = ['UP', 'DOWN', 'LEFT', 'RIGHT', 'STAY']
        self.n_actions = len(self.actions)
        self.reset()

    def reset(self):
        # Initial positions (Figure 6.1-like setup)
        # You can tweak starting coordinates if you have the figure
        self.pos_A = [2, 1]
        self.pos_B = [2, 3]
        self.ball_owner = random.choice(['A', 'B'])
        self.done = False
        return self._get_state()

    def _get_state(self):
        # State = (A_row, A_col, B_row, B_col, ball_owner)
        return (self.pos_A[0], self.pos_A[1],
                self.pos_B[0], self.pos_B[1],
                0 if self.ball_owner == 'A' else 1)

    def _move(self, pos, action):
        r, c = pos
        if action == 'UP':
            r = max(0, r - 1)
        elif action == 'DOWN':
            r = min(self.rows - 1, r + 1)
        elif action == 'LEFT':
            c = max(0, c - 1)
        elif action == 'RIGHT':
            c = min(self.cols - 1, c + 1)
        # STAY means do nothing
        return [r, c]

    def step(self, action_A, action_B):
        """Executes both players’ actions in random order."""
        if self.done:
            raise ValueError("Episode has ended. Call reset().")

        # Randomize execution order
        order = random.choice([('A', 'B'), ('B', 'A')])

        for agent in order:
            if agent == 'A':
                new_pos = self._move(self.pos_A, self.actions[action_A])
                # Collision logic
                if new_pos == self.pos_B:
                    if self.ball_owner == 'A':
                        self.ball_owner = 'B'  # Lose the ball
                    # Stay in place
                else:
                    self.pos_A = new_pos
            else:  # Agent B
                new_pos = self._move(self.pos_B, self.actions[action_B])
                if new_pos == self.pos_A:
                    if self.ball_owner == 'B':
                        self.ball_owner = 'A'
                else:
                    self.pos_B = new_pos

        reward_A, reward_B = 0, 0

        # Check goal conditions
        if self.ball_owner == 'A' and self.pos_A[1] == self.cols - 1:
            reward_A, reward_B = 1, -1
            self.done = True
        elif self.ball_owner == 'B' and self.pos_B[1] == 0:
            reward_A, reward_B = -1, 1
            self.done = True
        elif random.random() < self.draw_prob:
            # Draw termination
            reward_A = reward_B = 0
            self.done = True

        return self._get_state(), (reward_A, reward_B), self.done

    def render(self):
        grid = [['.' for _ in range(self.cols)] for _ in range(self.rows)]
        a_r, a_c = self.pos_A
        b_r, b_c = self.pos_B
        grid[a_r][a_c] = 'A*' if self.ball_owner == 'A' else 'A'
        grid[b_r][b_c] = 'B*' if self.ball_owner == 'B' else 'B'
        print("\n".join(" ".join(row) for row in grid))
        print(f"Ball: {self.ball_owner}\n")

# Example usage
if __name__ == "__main__":
    env = SoccerEnv()
    s = env.reset()
    done = False
    while not done:
        aA = np.random.randint(env.n_actions)
        aB = np.random.randint(env.n_actions)
        s, r, done = env.step(aA, aB)
        env.render()
        print("Reward:", r)


. . . . .
. . . B* .
A . . . .
. . . . .
Ball: B

Reward: (0, 0)
. . . B* .
A . . . .
. . . . .
. . . . .
Ball: B

Reward: (0, 0)
A . B* . .
. . . . .
. . . . .
. . . . .
Ball: B

Reward: (0, 0)
. B* . . .
A . . . .
. . . . .
. . . . .
Ball: B

Reward: (0, 0)
B* . . . .
. . . . .
A . . . .
. . . . .
Ball: B

Reward: (-1, 1)
